<a href="https://colab.research.google.com/github/skywalker0803r/works/blob/master/test_202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install catboost
from google.colab import drive
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
import numpy as np
drive.mount('/content/drive')
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge,LinearRegression,Lasso
from catboost import CatBoostRegressor
import seaborn as sns
import warnings;warnings.simplefilter('ignore')
import plotly.express as px
import joblib
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer,mean_squared_error
from mlxtend.regressor import StackingCVRegressor
from math import sqrt
from sklearn.ensemble import IsolationForest
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
import plotly.express as px

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# load data

In [2]:
data = pd.read_csv('/content/drive/My Drive/台塑輕油案子/data/test_202.csv',index_col=0).reset_index(drop=True)

#drop_idx = [201, 123, 124, 125, 126, 127,72,73,74,75,76,162,6,120,121,122,24,25]
#drop_idx = [24,25,72,73,74,75,76,123,124,125,126]
#data = data.drop(drop_idx,axis=0)

print(data.shape)
data.head()

(202, 27)


,T10,T50,T90,N+A,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
0,104.1,120.8,144.3,31.171088,0.16334,0.12310,0.02096,3.861608,1.406203,2.226804,0.396201,10.304644,7.407667,6.171137,2.929906,10.937803,12.122954,5.770600,5.984212,8.109563,9.448570,3.351409,3.904708,0.929217,4.014243,0.210378,0.204773
1,105.0,120.9,144.7,30.540117,0.04207,0.03195,0.00858,3.759808,1.224702,2.148804,0.381701,10.661744,7.619034,6.016270,2.873906,11.232064,12.414381,5.756414,5.730711,8.230962,9.665262,3.359719,3.886108,0.827859,3.750046,0.191193,0.186712
2,104.4,121.0,145.1,30.258878,0.03059,0.02415,0.00776,3.989816,1.404306,2.206509,0.390402,10.679802,7.652747,5.969249,2.863411,11.153468,12.199040,5.653507,5.601422,8.285277,9.562555,3.337253,3.866715,0.842654,3.916717,0.201739,0.160910
3,104.8,120.5,143.9,30.683788,0.03906,0.02690,0.01114,3.965321,1.378655,2.241277,0.397003,10.526617,7.494306,6.091945,2.891420,10.530323,11.521518,5.431952,5.988442,8.133103,11.030110,3.285644,3.978928,0.845741,3.824557,0.191976,0.174061
4,104.8,120.7,143.9,30.879431,0.07159,0.04725,0.01666,3.822696,1.335499,2.248498,0.391800,10.438747,7.358820,6.237209,2.859597,11.161902,12.234511,5.827158,5.723994,8.267642,9.554038,3.326798,3.883596,0.892058,3.935816,0.207975,0.156146


# check columns

In [3]:
cols = data.columns.tolist()
x_cols = cols[:4]
y_cols = cols[4:]
print(x_cols)
print(y_cols)

['T10', 'T50', 'T90', 'N+A']
['C5NP', 'C5IP', 'C5N', 'C6NP', 'C6IP', 'C6N', 'C6A', 'C7NP', 'C7IP', 'C7N', 'C7A', 'C8NP', 'C8IP', 'C8N', 'C8A', 'C9NP', 'C9IP', 'C9N', 'C9A', 'C10NP', 'C10IP', 'C10N', 'C10A']


# cv 10 for each y_name

In [4]:
X = data[x_cols]
rmse_scorer = make_scorer(lambda y_t,y_p : sqrt(mean_squared_error(y_t,y_p)))

results_dict = {}
for y_name in y_cols:
  #model = CatBoostRegressor(n_estimators = 100,learning_rate = 0.1,verbose=0)
  model = Pipeline([('scaler',StandardScaler()),('reg',Ridge())])
  results_dict[y_name] = cross_val_score(model, X, data[y_name], cv = 100 , scoring = rmse_scorer)

results = pd.DataFrame(results_dict).T
results['AVG'] = results.mean(axis=1)

results[['AVG']].apply(lambda x:round(x,2)).T.style.background_gradient(axis=1)

,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
AVG,0.21,0.19,0.04,0.3,0.23,0.37,0.17,0.34,0.41,0.98,0.33,0.35,0.5,0.82,1.01,0.26,0.41,0.29,0.27,0.15,0.29,0.07,0.23


In [5]:
results['AVG'].mean()

0.3567943641033471

# not pass y_list

In [6]:
not_pass_y_list = results.loc[results['AVG']>0.6].index.tolist()
not_pass_y_list

['C7N', 'C8N', 'C8A']

# outlier detecttion

In [0]:
# rename for cooks_d
data = data.rename(columns={'N+A':'N_add_A'})
x_cols = ['T10','T50','T90','N_add_A']
out_idx = []

for y_name in not_pass_y_list:
  for i,x_name in enumerate(x_cols):
    
    # use two variable caculate cooks_d
    m = ols(y_name + '~' + x_name ,data).fit()
    infl = m.get_influence()
    sm_fr = infl.summary_frame()
    data['cooks_d'] = sm_fr['cooks_d']
    
    # if cooks_d > ub(median + 3std) consider is outliers 
    ub = data['cooks_d'].median() + 3*data['cooks_d'].std()
    
    #append to out_idx
    out_idx = set(out_idx)|set(data.loc[data['cooks_d']>ub].index.tolist())
    out_idx = list(out_idx)

# tag_outliers

In [0]:
data['outlier_type'] = 'normal'

In [9]:
print(out_idx)

[201, 123, 124, 125, 126, 127]


In [0]:
data.loc[[201, 123, 124, 125, 126, 127],'outlier_type'] = 'high_cooksd'

In [0]:
#data.loc[[162,6,120,121,122],'outlier_type'] = 'Artificial_definition'

In [12]:
data['outlier_type'].value_counts()

normal         196
high_cooksd      6
Name: outlier_type, dtype: int64

# plot out_idx

In [13]:
data['index'] = data.index
for y_name in not_pass_y_list:
  for x_name in x_cols:
    fig = px.scatter(data,
                     x=x_name,
                     y=y_name,
                     color="outlier_type",
                     hover_data = ['index'])
    fig.show()

In [14]:
X = data.loc[data['outlier_type']=='normal'][x_cols]
y = data.loc[data['outlier_type']=='normal'][y_cols]
rmse_scorer = make_scorer(lambda y_t,y_p : sqrt(mean_squared_error(y_t,y_p)))

results_dict = {}
for y_name in y_cols:
  model = Pipeline([('scaler',StandardScaler()),('reg',Ridge())])
  results_dict[y_name] = cross_val_score(model, X, y[y_name], cv = 100 , scoring = rmse_scorer)

results = pd.DataFrame(results_dict).T
results['AVG'] = results.mean(axis=1)

results[['AVG']].apply(lambda x:round(x,2)).T.style.background_gradient(axis=1)

,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
AVG,0.17,0.14,0.05,0.26,0.21,0.32,0.07,0.28,0.41,0.73,0.3,0.29,0.42,0.64,0.39,0.25,0.38,0.23,0.26,0.14,0.24,0.07,0.19


# Artificial

In [0]:
data.loc[[24,25,72,73,74,75,76,162],'outlier_type'] = 'Artificial'

In [16]:
X = data.loc[data['outlier_type']=='normal'][x_cols]
y = data.loc[data['outlier_type']=='normal'][y_cols]
rmse_scorer = make_scorer(lambda y_t,y_p : sqrt(mean_squared_error(y_t,y_p)))

results_dict = {}
for y_name in y_cols:
  model = Pipeline([('scaler',StandardScaler()),('reg',Ridge())])
  results_dict[y_name] = cross_val_score(model, X, y[y_name], cv = 100 , scoring = rmse_scorer)

results = pd.DataFrame(results_dict).T
results['AVG'] = results.mean(axis=1)

results[['AVG']].apply(lambda x:round(x,2)).T.style.background_gradient(axis=1)

,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
AVG,0.16,0.13,0.04,0.23,0.2,0.3,0.04,0.25,0.36,0.59,0.22,0.25,0.39,0.59,0.27,0.22,0.37,0.21,0.24,0.14,0.24,0.07,0.19


In [17]:
data['outlier_type'].value_counts()

normal         188
Artificial       8
high_cooksd      6
Name: outlier_type, dtype: int64

# plot

In [18]:
data['index'] = data.index
for y_name in not_pass_y_list:
  for x_name in x_cols:
    fig = px.scatter(data,
                     x=x_name,
                     y=y_name,
                     color="outlier_type",
                     hover_data = ['index'])
    fig.show()